In [41]:
import os
parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)

####1
import pandas as pd
import numpy as np
import random

unified_columns = [f"{i}" for i in range(604)]
filename = os.path.join(parent_dir, 'Step 1 Data Processing/ROBOKOP+DrugMechDB/ROBOKOP+DrugmechDB Data/ROBOMechDB Processed Triples.csv')
df=pd.read_csv(filename)
df.columns = ['0','drug_name','disease_name','protein_name']
df = df.drop(['0'],axis=1)

drug_list = df.iloc[:,0].values.tolist()
disease_list = df.iloc[:,1].values.tolist()
protein_list = df.iloc[:,2].values.tolist()

no_overlap_positives = []
no_overlap_indices = []
for i in range(len(df)):
    drug_name = df.iloc[i,0]
    disease_name = df.iloc[i,1]
    protein_name = df.iloc[i,2]
    if drug_list.count(drug_name) == 1 and disease_list.count(disease_name) == 1 and protein_list.count(protein_name) == 1:
        no_overlap_positives.append([drug_name,disease_name,protein_name])
        no_overlap_indices.append(i)

print(len(no_overlap_positives))
print(no_overlap_indices)

df_external = df.iloc[no_overlap_indices]
print(df_external)

all_indices = set(df.index) 
internal_validation_indices = list(all_indices - set(no_overlap_indices))
df_internal = df.iloc[internal_validation_indices]
print(df_internal)

dictionary = {}
pos_values = my_list = [1] * len(df)
pos_keys = [df.iloc[i,0] + " " + df.iloc[i,1]+ " " + df.iloc[i,2] for i in range(0,len(df))]
for key,value in zip(pos_keys,pos_values):
    dictionary[key] = value

/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24
15
[24, 50, 62, 63, 798, 1615, 1706, 1937, 1981, 2118, 3046, 3880, 4102, 8597, 9470]
             drug_name                               disease_name  \
24                 fsh                    primary ovarian failure   
50    thyrotropin alfa                              thyroid tumor   
62          fomepizole                          acidosis disorder   
63       afamelanotide              erythropoietic protoporphyria   
798        teduglutide                       short bowel syndrome   
1615        nitisinone                                tyrosinemia   
1706        migalastat                              fabry disease   
1937        sivelestat                          acute lung injury   
1981     menatetrenone             hemorrhagic disease of newborn   
2118          glucagon                               hypoglycemia   
3046        eliglustat                            gaucher disease   
3880      pexidartinib        

In [42]:
unique_triples_drug = sorted(list(set(df_external['drug_name'])))
unique_triples_disease = sorted(list(set(df_external['disease_name'])))
unique_triples_protein = sorted(list(set(df_external['protein_name'])))

In [43]:
print(len(unique_triples_drug))
print(len(unique_triples_disease))
print(len(unique_triples_protein))

15
15
15


In [44]:
####2 Importing created protein, disease, and drug vector dictionaries into here

def csvtodict(filename):
    vector_dict = {}
    df_vector_dict = pd.read_csv(filename, header = 0)
    keys = df_vector_dict.iloc[:,0].tolist()
    values = list(range(len(df_vector_dict)))
    for key,value in zip(keys,values):
        vector_dict[key] = value
    return vector_dict

protein_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'))

disease_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'))

drug_dict = csvtodict(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'))

protein_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'),header = 0)
disease_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'),header = 0)
drug_df = pd.read_csv(os.path.join(parent_dir, 'Step 2 Data Embedding & Model Development/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'), header = 0)

In [45]:
###3 Building a 600 column vector array of all positive triples

positive_triples_vector_array = []
import itertools

for i in range(0,len(df_external)):
    try:
        drug_vector = drug_df.iloc[drug_dict[df_external.iloc[i,0]],1:201].tolist()
        disease_vector = disease_df.iloc[disease_dict[df_external.iloc[i,1]],1:201].tolist()
        protein_vector = protein_df.iloc[protein_dict[df_external.iloc[i,2]],1:201].tolist()
        
        drug_name = df_external.iloc[i,0]
        disease_name = df_external.iloc[i,1]
        protein_name = df_external.iloc[i,2]
        
        row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[1]]
        merged = list(itertools.chain(*row))
        positive_triples_vector_array.append(merged)

    except KeyError:
        continue

positive_triples_dataframe = pd.DataFrame(positive_triples_vector_array)


In [46]:
print(positive_triples_dataframe)

                 0                                          1    \
0                fsh                    primary ovarian failure   
1   thyrotropin alfa                              thyroid tumor   
2      afamelanotide              erythropoietic protoporphyria   
3        teduglutide                       short bowel syndrome   
4         nitisinone                                tyrosinemia   
5         migalastat                              fabry disease   
6         sivelestat                          acute lung injury   
7      menatetrenone             hemorrhagic disease of newborn   
8           glucagon                               hypoglycemia   
9         eliglustat                            gaucher disease   
10      pexidartinib              tenosynovial giant cell tumor   
11         mupirocin                                   impetigo   
12      l-citrulline  ornithine carbamoyltransferase deficiency   
13         ivacaftor                  pulmonary cystic fibrosi

In [47]:
###4 Use random combinations of drug, disease, protein present in positive triples to form negatives. Create negative vector array

negative_triples_array = []
negative_triples_vector_array = []
temp = set()
i=0

random.seed(42)

while i < 11.6*len(positive_triples_dataframe):  ###create negative triples
    drug = random.sample(unique_triples_drug,k=1)[0]
    protein = random.sample(unique_triples_protein, k=1)[0]
    disease = random.sample(unique_triples_disease,k=1)[0]
    key = drug + " " + disease + " " + protein
    if (key in dictionary)or (key in temp):
        print('true')
        continue
    temp.add(key)
    negative_triples_array.append([drug, disease, protein])
    i+= 1 

df_negative_triples = pd.DataFrame(negative_triples_array)
    
for i in range(0,len(df_negative_triples)):
    try:
        drug_vector = drug_df.iloc[drug_dict[df_negative_triples.iloc[i,0]],1:201].tolist()
        disease_vector = disease_df.iloc[disease_dict[df_negative_triples.iloc[i,1]],1:201].tolist()
        protein_vector = protein_df.iloc[protein_dict[df_negative_triples.iloc[i,2]],1:201].tolist()
        
        drug_name = df_negative_triples.iloc[i,0]
        disease_name = df_negative_triples.iloc[i,1]
        protein_name = df_negative_triples.iloc[i,2]
        
        row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[0]]
        merged = list(itertools.chain(*row))
        negative_triples_vector_array.append(merged)

    except KeyError:
        continue

negative_triples_dataframe = pd.DataFrame(negative_triples_vector_array)

df_randomNegatives = pd.concat([positive_triples_dataframe,negative_triples_dataframe],axis=0,ignore_index=True)
df_randomNegatives.columns = unified_columns
print(df_randomNegatives.columns)



df_randomNegatives.to_csv(os.path.join(parent_dir, 'Step 3 External Validation/External Validation Datasets/randomNegatives No Overlap External Validation Set.csv'), index=False)


true
true
true
true
true
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '594', '595', '596', '597', '598', '599', '600', '601', '602', '603'],
      dtype='object', length=604)


In [48]:
print(df_randomNegatives)

                    0                               1  \
0                 fsh         primary ovarian failure   
1    thyrotropin alfa                   thyroid tumor   
2       afamelanotide   erythropoietic protoporphyria   
3         teduglutide            short bowel syndrome   
4          nitisinone                     tyrosinemia   
..                ...                             ...   
150         ivacaftor               acute lung injury   
151  thyrotropin alfa  hemorrhagic disease of newborn   
152          glucagon                        impetigo   
153       teduglutide            short bowel syndrome   
154          glucagon   tenosynovial giant cell tumor   

                                            2        3         4         5  \
0       follicle stimulating hormone receptor  0.86115  0.698690 -0.187320   
1        thyroid stimulating hormone receptor  0.78651  1.439200  0.593200   
2                     melanocortin 1 receptor  0.96977 -0.065942  0.188310   
3  

In [54]:
#####1.

import pandas as pd
import os
parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)


df_randNegs = pd.read_csv(os.path.join(parent_dir, 'Step 3 External Validation/External Validation Datasets/randomNegatives No Overlap External Validation Set.csv'))

df_randNegs.drop(['0'],axis=1)


unified_columns = [f"{i}" for i in range(604)]


values = my_list = [1] * len(df_randNegs)
keys = [df_randNegs.iloc[i,0] + " " + df_randNegs.iloc[i,1]+ " " + df_randNegs.iloc[i,2] for i in range(0,len(df_randNegs))]
for key,value in zip(keys,values):
    dictionary[key] = value


####Find all unique drugs, diseases, and proteins in this 80% set. 
unique_triples_drug = sorted(list(set(df_internal['drug_name'])))
unique_triples_disease =sorted(list(set(df_internal['disease_name'])))
unique_triples_protein = sorted(list(set(df_internal['protein_name'])))


/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24


In [57]:
###3 Complete Set 1 Development

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_curve
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools
import random

random_seeds = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,42]

model_auc_values = []

for i in random_seeds:
    random.seed(i)
    positive_triples_vector_array = []
    for j in range(0,len(df_internal)):
        try:
            drug_vector = drug_df.iloc[drug_dict[df_internal.iloc[j,0]],1:201].tolist()
            disease_vector = disease_df.iloc[disease_dict[df_internal.iloc[j,1]],1:201].tolist()
            protein_vector = protein_df.iloc[protein_dict[df_internal.iloc[j,2]],1:201].tolist()
        
            drug_name = df_internal.iloc[j,0]
            disease_name = df_internal.iloc[j,1]
            protein_name = df_internal.iloc[j,2]
        
            row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[1]]
            merged = list(itertools.chain(*row))
            positive_triples_vector_array.append(merged)

        except KeyError:
            continue

    positive_triples_dataframe = pd.DataFrame(positive_triples_vector_array)
    negative_triples_array = []
    negative_triples_vector_array = []
    temp = set()
    j=0

    while j < int(1.1*len(positive_triples_dataframe)):  ###create negative triples
        drug = random.sample(unique_triples_drug,k=1)[0]
        protein = random.sample(unique_triples_protein, k=1)[0]
        disease = random.sample(unique_triples_disease,k=1)[0]
        if (drug + " " + disease + " " + protein) in dictionary or (drug + " " + disease + " " + protein) in temp:
            continue
        temp.add(drug + " " + disease + " " + protein)
        negative_triples_array.append([drug, disease, protein])
        j+= 1 

    df_negative_triples = pd.DataFrame(negative_triples_array)
    j=0
    for j in range(0,len(df_negative_triples)):
        try:
            drug_vector = drug_df.iloc[drug_dict[df_negative_triples.iloc[j,0]],1:201].tolist()
            disease_vector = disease_df.iloc[disease_dict[df_negative_triples.iloc[j,1]],1:201].tolist()
            protein_vector = protein_df.iloc[protein_dict[df_negative_triples.iloc[j,2]],1:201].tolist()
        
            drug_name = df_negative_triples.iloc[j,0]
            disease_name = df_negative_triples.iloc[j,1]
            protein_name = df_negative_triples.iloc[j,2]
        
            row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[0]]
            merged = list(itertools.chain(*row))
            negative_triples_vector_array.append(merged)

        except KeyError:
            continue

    negative_triples_dataframe = pd.DataFrame(negative_triples_vector_array)
    df = pd.concat([positive_triples_dataframe,negative_triples_dataframe],axis=0,ignore_index=True)
    df.columns = unified_columns
    test = pd.concat([df_randNegs, df],axis=0)
    col = ['0', '1', '2']
    print(len(test))
    rem = test.drop_duplicates(subset = col)
    print(len(rem))

# Assuming df is already defined and contains your dataset
    
# Assuming df is already defined and contains your dataset
    shuffled_df = df.sample(frac=1.0, random_state=i) 
    data = shuffled_df.iloc[:, 3:-1].values
    labels = shuffled_df.iloc[:, -1].values
    print(shuffled_df)
    num_folds = 5
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

    all_predictions = []
    all_test_sets = []

    # Perform 5-fold cross-validation to test the accuracy of the model
    fold = 1
    auc_values = []

    tf.keras.utils.set_random_seed(i)

    for train_index, test_index in skf.split(data, labels):
        print(f"Fold: {fold}")
    
    # Split the data into training and test sets
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        tf_train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
        tf_test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))
        
        BATCH_SIZE = 32
        STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
        tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
        tf_test_set = tf_test_set.batch(BATCH_SIZE)

    # Define the learning rate schedule
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
            0.001,
            decay_steps=STEPS_PER_EPOCH * 1000,
            decay_rate=1,
            staircase=False
        )
        
        
        def get_optimizer():
            return tf.keras.optimizers.Adam(lr_schedule)

        optimizer = get_optimizer()
        
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape=(600,)),
            tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)), 
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

        # Implement early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
        # Train the model
        model.fit(tf_train_set,
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=40,
                    validation_data = tf_test_set, 
                    callbacks=[early_stopping], 
                    verbose=0)
        
        predictions = model.predict(X_test)
        fpr,tpr, thresholds = roc_curve(y_test,predictions)
        roc_auc = auc(fpr, tpr)
        auc_values.append(roc_auc)
        print("AUROC for test data:", roc_auc)
        all_predictions.append(predictions)
        all_test_sets.append(y_test)
        
        ####Before each iteration ends, save each model###
        
        save_model= tf.keras.models.save_model(model, os.path.join(os.getcwd(), f'Classification Models/Complete Set 1 No Overlap Models/ROBOMechDB Complete Set 1 No Overlap Model Seed {i}.keras'))

        fold += 1
        

    fcv_mean_auc = np.mean(auc_values)
    fcv_std_auc = np.std(auc_values)
    fcv_min_auc = np.min(auc_values)
    fcv_max_auc = np.max(auc_values)
    print(fcv_mean_auc, fcv_min_auc,fcv_max_auc)
    model_auc_values.append([fcv_min_auc,fcv_max_auc,fcv_mean_auc,fcv_std_auc])

fcv_df = pd.DataFrame(model_auc_values, columns=['min_auc', 'max_auc', fcv_mean_auc,fcv_std_auc])

min_auc = np.min(fcv_df.iloc[:,0])
max_auc = np.max(fcv_df.iloc[:,1])
avg_auc = np.mean(fcv_df.iloc[:,2])
std_dev_auc = np.mean(fcv_df.iloc[:,3])

ensemble_stats = np.array([min_auc,max_auc,avg_auc,std_dev_auc])
ensemble_stats = ensemble_stats.reshape(1, -1)
ensemble_stats_df = pd.DataFrame(ensemble_stats, columns = ['min_auc', 'max_auc', 'avg_auc', 'std_dev_auc'])
ensemble_stats_df.to_csv(os.path.join(os.getcwd(), 'Classification Models/Complete Set 1 No Overlap Models/no_overlap_complete_set_1_ensemble_stats.csv'))


18520
18520
                    0                            1  \
16396     defibrotide           granuloma annulare   
16915     candesartan                         acne   
17596       biperiden                 endometritis   
6084     progesterone                endometriosis   
9456        pazopanib              stroke disorder   
...               ...                          ...   
10955  spironolactone     dermatitis herpetiformis   
17289   apraclonidine  interstitial lung disease 2   
5192   spironolactone        hypertensive disorder   
12172       ezetimibe      diffuse alopecia areata   
235       regorafenib     hepatocellular carcinoma   

                                                       2         3         4  \
16396                   nuclear factor kappa b subunit 1  0.578510 -0.194070   
16915                    ebp cholestenol delta-isomerase  0.612560  0.188450   
17596                                  cdc like kinase 2  0.326680  0.561680   
6084               